# Train frame-wise phone classifier

2018-08-22

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np

import tensorflow as tf
import numdifftools as nd
from sklearn.model_selection import train_test_split
from train import *

## Prepare data

In [2]:
fftlin = np.load('fft_linfilt_pca_reduced.npy')
pca = np.load('fft_linfilt_pca.npy').item()
labels = np.load('labels.npy')
phn2idx = np.load('phn2idx.npy').item()
idx2phn = np.load('idx2phn.npy').item()

In [3]:
# Divide train/test set
X_train, X_test, Y_train, Y_test = train_test_split(fftlin, labels, test_size=0.2)

In [4]:
X_train.shape, Y_train.shape

((31021, 40), (31021,))

In [5]:
X_test.shape, Y_test.shape

((7756, 40), (7756,))

### Convert to idx

In [6]:
Y_train_onehot = np.array([phn2idx[phn] for phn in Y_train])
Y_test_onehot = np.array([phn2idx[phn] for phn in Y_test])

In [7]:
max_epoch = 500
n = 50 # n_display
uq_vowels = list(set(labels))

# Train
total_cost, total_test, moving_avg, final_epoch, params = train_nn(
    (X_train, Y_train_onehot), (X_test, Y_test_onehot), uq_vowels, 
    training_epochs=max_epoch, 
    model_dir=os.path.join('model','test'), n_display=n, verbose=True)

INFO:train:learning_rate: 0.01
INFO:train:training_epochs: 500
INFO:train:n_display: 50
INFO:train:train_x: (31021, 40), train_y: (31021,)
INFO:train:test_x: (7756, 40), test_y: (7756,)

INFO:train:hidden_output_size: 500
INFO:train:final_output_size: 9

INFO:train:  50/500 train: CE=0.9070, test: CE=0.9475, test_acc:0.64 slop=0.94753
INFO:train: 100/500 train: CE=0.7081, test: CE=0.8670, test_acc:0.68 slop=-0.08053
INFO:train: 150/500 train: CE=0.6189, test: CE=0.9026, test_acc:0.68 slop=0.03559
INFO:train: 200/500 train: CE=0.6291, test: CE=0.9474, test_acc:0.67 slop=0.04482
INFO:train: 250/500 train: CE=0.5218, test: CE=1.0088, test_acc:0.67 slop=0.06143
INFO:train: 300/500 train: CE=0.4732, test: CE=1.0726, test_acc:0.66 slop=0.06379
INFO:train: 350/500 train: CE=0.3949, test: CE=1.1664, test_acc:0.67 slop=0.09374
INFO:train: 400/500 train: CE=0.4009, test: CE=1.3827, test_acc:0.64 slop=0.21631
INFO:train: 450/500 train: CE=0.3643, test: CE=1.2176, test_acc:0.66 slop=-0.16505
INFO:

## 해야할것

- 훈련 accuracy도 추가하기
- hidden size를 늘리고 dropout을 추가하자!  
- batch norm도 추가하기
- class 형태로 코드 정리하기 (trainmode testmode가 가능하도록)
- 테스트 코드 작성하기